In [1]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

In [16]:
import loader as load

In [2]:
SPARK_CONFIG

{'kerberos_auth': True,
 'numofcores': 9,
 'numofinstances': 9,
 'process_label': 'RECSYS_STORY',
 'replication_num': 1}

# Spark 

In [3]:
sp = spark(**SPARK_CONFIG)
hive = sp.sql

# Cheking oracle table 

In [17]:
ld=load.Loader(init_dsn=True, encoding='cp1251',  sep=',')
db = load.OracleDB('', '', ld._get_dsn('')) 
db.connect()

In [18]:
db

In [19]:
table_name = 'recsys_story_asup_ratings'

In [20]:
ISKRA_TABLE = table_name

query = '''
select /*+ parallel (8) */
          *
FROM {}
where inn is not Null
and rownum <= 10000
'''.format(ISKRA_TABLE)
df = pd.read_sql(query, con=db.connection)

In [88]:
db.connection

<cx_Oracle.Connection to tech_iskra[iskra]@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=10.112.79.164)(PORT=1521))(CONNECT_DATA=(SID=iskra4)))>

In [ ]:
df.head()

In [22]:
query = '''
select max(LOAD_DT) AS MAX_DATE
FROM {}
where inn is not Null
'''.format(ISKRA_TABLE)

In [23]:
max_date_pd = pd.read_sql(query, con=db.connection)

In [24]:
max_date_pd.head()

,MAX_DATE
0,2021-10-05


In [32]:
'2021-09-23'

'2021-09-23'

In [28]:
max_date_load = max_date_pd["MAX_DATE"][0]

In [29]:
max_date_load

'2021-10-05'

In [37]:
max_date_load = '2021-09-23'

In [38]:
query_for_inn_cnt = """
SELECT count(INN) AS INN_CNT
FROM {}
WHERE inn is not Null
  and LOAD_DT = '{}'
""".format(ISKRA_TABLE, max_date_load)

In [43]:
print(query_for_inn_cnt)


SELECT count(distinct INN) AS INN_CNT_DIST
FROM recsys_story_asup_ratings
WHERE inn is not Null
  and LOAD_DT = '2021-09-23'



In [39]:
inn_cnt_dist_pd = pd.read_sql(query_for_inn_cnt, con=db.connection)

In [40]:
inn_cnt_dist = inn_cnt_dist_pd["INN_CNT"][0]

In [41]:
inn_cnt_dist

0

In [48]:
max_date_load

'2021-09-23'

In [16]:
np.unique(df['LOAD_DT'])

array(['2021-09-23'], dtype=object)

# mail sendler 

In [50]:
from lib.mail_sender import *
import json

In [51]:
EMAIL_SETTINGS_FILE = "mail_settings/mail_settings.txt"

In [52]:
with open(EMAIL_SETTINGS_FILE, "r") as f:
    settings = json.load(f)

In [59]:
EMAIL_LIST_FILE = "mail_settings/mail_list.txt"

In [77]:
with open(EMAIL_LIST_FILE, "r") as f:
    recivers_str = f.read()
    receivers = recivers_str.split('\n')

In [78]:
receivers

['EIShubochkin@omega.sbrf.ru', 'EIShubochkin@sberbank.ru']

In [80]:
msg = """
ISKRA:
Последняя дата загрузки: {}
Число уникальных ИНН на последнюю дату: {}
""".format(max_date_load, inn_cnt_dist)

In [81]:
msg

'\nISKRA:\nДата загрузки: 2021-09-23\nЧисло уникальных ИНН: 1184714\n'

In [67]:
authorization = Authorization(user=settings["user"], domain='ALPHA', mailbox=settings["mailbox"],
                              server='cab-vop-mbx2053.omega.sbrf.ru')

In [68]:
mail_receiver = MailReceiver(settings["password"], auth_class=authorization)

In [74]:
date = datetime.strftime(datetime.today(), format='%Y-%m-%d')

In [79]:
mail_receiver.send_message(receivers, 'SBBOL_STORY_RATINGS_STATS {}'.format(date), msg)

# Cheking dataset 

In [12]:
dataset = load_table(SBX_TEAM_DIGITCAMP, ALS_HISTORY_ASUP_RATINGS, hive).cache()

In [14]:
# drop_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET_POSITIVE, hive)

In [ ]:
# drop_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET_POSITIVE, hive)

In [15]:
%%time
# hive.sql("drop table if exists {schema}.{tbl} purge".format(schema=SBX_TEAM_DIGITCAMP, tbl=RECSYS_STORY_DATASET))

CPU times: user 2.65 ms, sys: 1.61 ms, total: 4.26 ms
Wall time: 1.74 s


DataFrame[]

In [ ]:
%%time
dataset.show()

In [15]:
%%time
dataset.select("inn").distinct().count()

CPU times: user 133 ms, sys: 156 ms, total: 289 ms
Wall time: 13min 27s


1189687

In [42]:
dataset.filter("load_dt = '2021-10-05'").select("inn").distinct().count()

1189687

In [43]:
dataset.filter("load_dt = '2021-09-23'").select("inn").distinct().count()

0

# Checking create date 

In [49]:
hive.sql("describe formatted {}.{}".format(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET_TARGET)).show()

,col_name,data_type,comment
0,user_id,int,None
1,item_id,int,None
2,timestamp,bigint,None
3,rating,double,None
4,,,
5,# Detailed Table Information,,
6,Database,sbx_team_digitcamp,
7,Table,recsys_story_dataset_target,
8,Owner,shubochkin1-ei_ca-sbrf-ru@DF.SBRF.RU,
9,Created Time,Tue Oct 05 15:24:06 MSK 2021,


In [ ]:
from py4j.protocol import Py4JJavaError

In [3]:
table_info = hive.sql("show create table {}.{}".format(SBX_TEAM_DIGITCAMP, ALS_ASUP_PIVOT)).toPandas().createtab_stmt[0]

In [10]:
target_dataset = load_table(SBX_TEAM_DIGITCAMP, ALS_ASUP_PIVOT, hive).cache()

In [8]:
%%time
target_dataset.select("inn").distinct().count()

CPU times: user 2.38 ms, sys: 5.42 ms, total: 7.8 ms
Wall time: 8.03 s


1189687

In [11]:
%%time
target_dataset.select("inn").distinct().count()

CPU times: user 93.2 ms, sys: 120 ms, total: 213 ms
Wall time: 10min 6s


1189687

In [9]:
target_dataset.select("item_id").distinct().count()

87

In [6]:
target_dataset.show()

,user_id,item_id,timestamp,rating
0,1,382389508,None,0.0
1,8,382545223,None,0.0
2,12,382364619,None,0.0
3,14,382391476,None,0.0
4,29,382393978,None,0.0
5,41,382348021,None,0.0
6,58,382364633,None,0.0
7,62,382393978,None,0.0
8,66,382545223,None,0.0
9,67,382567886,None,0.0
